# Pixelbuffer Graph

A graph that accepts two pixel buffers normalized to `[0,1]` values and combines them as though it were exponentially decaying a new input and a previous input. The output is a weighted pixel buffer of the same dimensions also normalized to `[0,1]`, which will need to be de-normalized back to a normal pixel buffer.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
X = tf.placeholder(tf.float32, shape=(224,224,3), name="input_x") # new input
Y = tf.placeholder(tf.float32, shape=(224,224,3), name="input_y") # previous input

X_weighted = tf.multiply(X, 0.1)
Y_weighted = tf.multiply(Y, 0.9)

Z = tf.add(X_weighted, Y_weighted, name="output_z")

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    z_eval = sess.run([Z], feed_dict={
        X: np.full([224,224,3], 0.75, dtype=np.float32), 
        Y: np.full([224,224,3], 0.25, dtype=np.float32)
    })
    
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input_x": X, "input_y": Y}, outputs={"output_z": Z})
    
    # print(z_eval)
    

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb


In [ ]:
# Test Code

sess = tf.InteractiveSession()

A = tf.fill([224,224,3], 0.75)
B = tf.fill([224,224,3], 0.25)

A_weighted = tf.multiply(A, 0.1)
B_weighted = tf.multiply(B, 0.9)

C = tf.add(A_weighted, B_weighted)

print(C.eval())

sess.close()

## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output_z
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input_x,input_y \
	--input_shapes=224,224,3:224,224,3 \
	--output_arrays=output_z
```